### Add model, connection and device to settings:

In [1]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings
from ddeint import ddeint

# пути и названия файлов
modelFileName = 'problems/advection_equation1'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:block_main.block_io:DefaultBound not set
INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


### Run solver:

In [2]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
progress = ProgressNotebook(100, prefix='solving')
from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))
def run(event):
    remoteProjectRun(settings, 1, progress)
bTest.on_click(run)
display(progress.progress)
remoteProjectRun(settings, 1, progress)
#display(bTest)

IntProgress(value=0, description='solving: ', max=99)

INFO:remoterun:
connect ...
INFO:remoterun:
connection established
INFO:remoterun:
files/folders routine
INFO:remoterun:
files/folders routine completed
INFO:remoterun:
Running preprocessor:
INFO:remoterun:command:
INFO:remoterun:~/anaconda3/bin/python3 -c "import hybriddomain.gens.hs.tests.tests_gen_1d as ts; ts.run()" -t /acchome/imaslenikov/tracer/workspace/problems/advection_equation1 -d /acchome/imaslenikov/tracer/workspace/settings/device_conf/devices.json -p /acchome/imaslenikov/tracer/workspace/settings/paths/connection.json -w /acchome/imaslenikov/tracer/workspace -u imaslenikov 2>&1
INFO:remoterun:
Running solver:
INFO:remoterun:01-16 14:16:08 Performance (10^6): 4.46293

INFO:remoterun:INFO:postproc.video:Creating images for block 0 value U

INFO:remoterun:INFO:postproc.video:Creating video file: 0

INFO:remoterun:INFO:postproc.video:Creating out file: u

INFO:remoterun:Downloading results...
INFO:remoterun:Done!


### Results:

In [3]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['u'])


### Video:

In [4]:
# available plots:
model.plots_paths

{'U': ['problems/advection_equation1/out/U.mp4']}

In [5]:
display(HTML(model.get_video('U')))

### U(x, t-fixed),V(x, t-fixed):

In [8]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']
print(result)

def f(t):
    result_x = np.array([result[time][0][t] for time in times])
    result_abs = np.array([math.sin(t/(len(result[0][0])-1)*3.1415926+time)   for time in times])
    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(time,result_x)
    # Круг на котором лежат начала траекторий 
    plt.plot(time, result_abs)
w = interactive(f,t=(0,len(result[0][0])-1,1))
display(w)

plt.show()

{0.0: array([[0.00000000e+00, 9.99983307e-03, 1.99986659e-02, 2.99955010e-02,
        3.99893336e-02, 4.99791689e-02, 5.99640049e-02, 6.99428469e-02,
        7.99146965e-02, 8.98785517e-02, 9.98334140e-02, 1.09778300e-01,
        1.19712204e-01, 1.29634142e-01, 1.39543116e-01, 1.49438128e-01,
        1.59318209e-01, 1.69182345e-01, 1.79029569e-01, 1.88858896e-01,
        1.98669329e-01, 2.08459899e-01, 2.18229622e-01, 2.27977529e-01,
        2.37702623e-01, 2.47403964e-01, 2.57080555e-01, 2.66731441e-01,
        2.76355654e-01, 2.85952210e-01, 2.95520216e-01, 3.05058628e-01,
        3.14566553e-01, 3.24043036e-01, 3.33487093e-01, 3.42897803e-01,
        3.52274239e-01, 3.61615419e-01, 3.70920479e-01, 3.80188406e-01,
        3.89418334e-01, 3.98609340e-01, 4.07760441e-01, 4.16870803e-01,
        4.25939471e-01, 4.34965521e-01, 4.43948120e-01, 4.52886283e-01,
        4.61779177e-01, 4.70625877e-01, 4.79425550e-01, 4.88177240e-01,
        4.96880144e-01, 5.05533338e-01, 5.14136016e-01, 5.

interactive(children=(IntSlider(value=157, description='t', max=314), Output()), _dom_classes=('widget-interac…

In [18]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']


def f(t):
    result_x = np.array([result[time][t][0] for time in times])
    result_abs = np.array([math.sin(t/(len(result[0])-1)*3.1415926+time)   for time in times])
     
    result_raz=np.array([abs(result_x[i]-result_abs[i])    for i in range(len(times))])
    #print(rezult_raz)
    #result_raz=np.array([abs(result[time][t][0]-math.sin(t/(len(result[0])-1)*3.1415926)*math.exp(-time)) for time in times])
    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(time,result_raz)
    # Круг на котором лежат начала траекторий 
    #plt.plot(time, result_abs)
w = interactive(f,t=(0,len(result[0])-1,1))
display(w)

plt.show()

interactive(children=(IntSlider(value=157, description='t', max=314), Output()), _dom_classes=('widget-interac…

In [10]:
# отрисовывает результат
%matplotlib inline

#display(w)
time = model.results_arrays['u']['timevalues']
#print(time)

times = model.results_arrays['u']['timevalues']
#print(times)
result = model.results_arrays['u']['resvalues']


def f(t):
    result_x = np.array([result[time][t][0] for time in times])
    result_abs = np.array([math.sin(t/(len(result[0])-1)*3.1415926+time)   for time in times])
    
    result_raz=np.array([abs((result_x[i]-result_abs[i])/result_abs[i])    for i in range(len(times))])
    #print(rezult_raz)
    #result_raz=np.array([abs(result[time][t][0]-math.sin(t/(len(result[0])-1)*3.1415926)*math.exp(-time)) for time in times])
    # Сама траектория зависимость от t берется разное начало запуска
    plt.plot(time,result_raz)
    # Круг на котором лежат начала траекторий 
    #plt.plot(time, result_abs)
w = interactive(f,t=(0,len(result[0])-1,1))
display(w)

plt.show()

interactive(children=(IntSlider(value=157, description='t', max=314), Output()), _dom_classes=('widget-interac…